#### Packages

In [1]:
import twitter
import pymongo
import csv
import pandas as pd
import sys
import time

#### Functions

In [29]:
# SEARCHER
# credentials: "coffee1"
########################################################
def twitter_search(HASHTAG, len_results):
    
    CONSUMER_KEY = 'cOExq4T3uYu9CbdqmMUukGVqV'
    CONSUMER_SECRET = 'u9WCLUaWCLyILtHNGDAMmowxoQseQenyW0avsXhuz8NkPgLHMv'
    OAUTH_TOKEN = '4061210361-uHZhAHfTim8nnyyCWeZXMfLbypbq3Z2jgGcTmhu'
    OAUTH_TOKEN_SECRET = 'nbC4YzeS7vBBBcOWyYgr4msKdLgfRznFQAon1ji2lJn2F'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
        CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    print >> sys.stderr, 'Succesfully connected to Twitter SEARCH API ... ... '
    print >> sys.stderr, 'Searching for \'%s\' ... ... ' %HASHTAG
    
    max_results= min(5000, len_results)
    
    search_results = twitter_api.search.tweets(q=HASHTAG, count=100, language='en')
    
    statuses = search_results['statuses']
    
    for _ in range(179):
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError:
            print "Found \t %i \t statuses." %len(statuses) 
            break
            
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        kwargs['language'] = 'en'
        
        search_results = twitter_api.search.tweets(**kwargs)

        statuses.extend(search_results['statuses'])
        
        if len(statuses) >= max_results: 
            break
        
    return statuses[0:len_results]


# STREAMER
# credentials: "searcher_nv_1"
########################################################
def twitter_stream(HASHTAG, len_results):

    CONSUMER_KEY = 'cOExq4T3uYu9CbdqmMUukGVqV'
    CONSUMER_SECRET = 'u9WCLUaWCLyILtHNGDAMmowxoQseQenyW0avsXhuz8NkPgLHMv'
    OAUTH_TOKEN = '4061210361-uHZhAHfTim8nnyyCWeZXMfLbypbq3Z2jgGcTmhu'
    OAUTH_TOKEN_SECRET = 'nbC4YzeS7vBBBcOWyYgr4msKdLgfRznFQAon1ji2lJn2F'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
        CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)

    twitter_stream = twitter.TwitterStream(auth=twitter_api.auth)
    
    print >> sys.stderr, 'Succesfully connected to Twitter STREAMING API ... ... '
    print >> sys.stderr, 'Streming for \'%s\' ... ... ' %HASHTAG
    
    stream = twitter_stream.statuses.filter(track=HASHTAG, language='en')
    
    statuses = []
    
    t_end = time.time()+60
    
    for status in stream:
        print status['text']
        print
        statuses.append(status)
        if time.time()>=t_end:
            print "Time elapsed ... ..."
            break

    return statuses

####################################################################################################
# SEARCH AND STREAM
def search_and_stream(HASHTAG, len_results):
    
    statuses_search = twitter_search(HASHTAG, len_results)
    
    print "Sleeping for 16 seconds ... ..."
    print "Start : %s" % time.ctime()
    time.sleep(16)
    print "End : %s" % time.ctime()

    stream = twitter_stream(HASHTAG, len_results)
    
    statuses_search += stream

    return statuses_search

####################################################################################################

def save_track_to_mongo(statuses, track_string, db, coll):
    
    client = pymongo.MongoClient()
    db = client.db
    collection = db.coll
    
    for tweet_doc in statuses:
        try:
            # 2 small modificatio to the tweet as it is gathered from API
            # 1. Add the tags of the search, to know from which search query the document has been obtained
            tweet_doc['track'] = track_string  
            # 2. Substitue the defoult _id assigned by Mongo with the tweet id to prevent the same tweet to be inserted more than once
            tweet_doc['_id'] = tweet_doc["id"]
            del tweet_doc['id']
            mongo_db_coll.insert_one(tweet_doc)
            print "Saving tweet in track %s" %track_string
        except:
            print "Error message from MongoDB, skipping tweet and continuing."
            

#### Collects a number of tweets for each hashtag in the track

In [30]:
track0_list_hashed = ['#ecig', '#vaping', '#vape', '#vaporizer', '#marijuana', '#sorrynotsorry', '#eliquid', '#ecigs', '#vapeporn', '#cannabis']
    
print track0_list_hashed


track0_statuses = []

for tag in track0_list_hashed:

    statuses = search_and_stream(tag, 300)
    track0_statuses += statuses
    
print "TOTAL number of statuses sampled:%i"%len(track0_statuses)


client = pymongo.MongoClient()
db = client.sn_sp
collection = db.net_0

for tweet_doc in track0_statuses:
    try:
        tweet_doc['track'] = 'track0'
        tweet_doc['_id'] = tweet_doc["id"]
        del tweet_doc['id']
        print "Saving tweet."
        collection.insert_one(tweet_doc)
    except:
        print "Preventing duplicate, skipping tweet and continuing."
        
client = pymongo.MongoClient()
collection1 = db.net_0
collection2 = db.net_00

cursor = collection1.find({'track':'track0'}, {'_id':'true', 'text':'true', 'entities.hashtags.text':'true', 'retweet_count':'true', 'track':'true'})

items_list = [ item for item in cursor ] 

for tweet_doc in items_list:
    hashtags = []
    for h in tweet_doc['entities']['hashtags']:
        h_lowered = h['text'].lower()
        hashtags.append(h_lowered)
    tweet_doc['hashtags'] = hashtags
    del tweet_doc['entities']
    try:
        print "Transfering tweet."
        collection2.insert_one(tweet_doc)
    except:
        print "Preventing duplicate, skipping tweet and continuing."

['#ecig', '#vaping', '#vape', '#vaporizer', '#marijuana', '#sorrynotsorry', '#eliquid', '#ecigs', '#vapeporn', '#cannabis']


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#ecig' ... ... 


Found 	 81 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:15:21 2016
End : Tue Aug 30 10:15:37 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vaping' ... ... 


Found 	 190 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:15:39 2016
End : Tue Aug 30 10:15:55 2016


...

Found 	 95 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:17:30 2016
End : Tue Aug 30 10:17:46 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vapeporn' ... ... 


Found 	 200 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:17:48 2016
End : Tue Aug 30 10:18:04 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#cannabis' ... ... 


Found 	 200 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:18:06 2016
End : Tue Aug 30 10:18:22 2016
TOTAL number of statuses sampled:1759
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tw

In [31]:
df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track1.csv') ###
track1_list = df['x'].tolist() ###
track1_list_hashed = [] ###
for h in track1_list: ###
    h = '#'+h
    track1_list_hashed.append(h) ###
    
print track1_list_hashed ###

track1_statuses = [] ###

for tag in track1_list_hashed: ###

    statuses = search_and_stream(tag, 300)
    track1_statuses += statuses ###
    
print "TOTAL number of statuses sampled:%i"%len(track1_statuses) ###

client = pymongo.MongoClient()
db = client.sn_sp
collection = db.net_0

for tweet_doc in track1_statuses: ###
    tweet_doc['track'] = 'track1' ###
    tweet_doc['_id'] = tweet_doc["id"]
    del tweet_doc['id']
    try:
        print "Saving tweet."
        collection.insert_one(tweet_doc)
    except:
        print "Preventing duplicate, skipping tweet and continuing."
        
client = pymongo.MongoClient()
collection1 = db.net_0
collection2 = db.net_00
                                    ###
cursor = collection1.find({'track':'track1'}, {'_id':'true', 'text':'true', 'entities.hashtags.text':'true', 'retweet_count':'true', 'track':'true'})

items_list = [ item for item in cursor ] 

for tweet_doc in items_list:
    hashtags = []
    for h in tweet_doc['entities']['hashtags']:
        h_lowered = h['text'].lower()
        hashtags.append(h_lowered)
    tweet_doc['hashtags'] = hashtags
    del tweet_doc['entities']
    try:
        print "Transfering tweet."
        collection2.insert_one(tweet_doc)
    except:
        print "Preventing duplicate, skipping tweet and continuing."

['#vaper', '#vapor', '#\xe3\x83\x95\xe3\x83\xac\xe3\x83\xbc\xe3\x83\x90\xe3\x83\xbc', '#\xe9\x9b\xbb\xe5\xad\x90\xe3\x82\xbf\xe3\x83\x90\xe3\x82\xb3', '#ejuice', '#vapelife', '#vapefam', '#vapotage', '#vapoteurs', '#ecigarette', '#vapecommunity', '#vapelyfe', '#vapeon', '#subohm', '#tobaccocontrol', '#alert', '#free', '#giveaway', '#vapenation', '#vapeshop', '#vapehooligans', '#smoking', '#dabs', '#notblowingsmoke', '#kandypens', '#legalizeit', '#vapepen', '#mmj', '#pot', '#weed', '#cbd', '#sativa', '#israel', '#investing', '#beverlyhills', '#queen', '#mme', '#vmas', '#doinghappysnoopydance', '#itsmyfriday', '#aldubsilanabaay', '#40likes', '#class', '#country', '#quoteoftheday', '#sand', '#teen', '#timylovebegins', '#oops', '#teacher5adayslowchat', '#blessed', '#mod', '#vapeuk', '#ukvape', '#calivapers', '#cloudchaser', '#coil', '#smokefree', '#quitsmoking', '#cloudchasing', '#ecigarettes', '#fda', '#reddit', '#instavape', '#coilporn', '#legalize', '#hemp', '#stoner', '#breakthetaboo',

Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vaper' ... ... 


Found 	 200 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:22:17 2016
End : Tue Aug 30 10:22:33 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vapor' ... ... 


Found 	 200 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:22:35 2016
End : Tue Aug 30 10:22:51 2016


...

Found 	 80 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:43:36 2016
End : Tue Aug 30 10:43:52 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#marijuanastocks' ... ... 


Found 	 132 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:43:53 2016
End : Tue Aug 30 10:44:09 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#weedstocks' ... ... 


Found 	 92 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:44:10 2016
End : Tue Aug 30 10:44:26 2016
TOTAL number of statuses sampled:11597
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing duplicate, skipping tweet and continuing.
Saving tweet.
Preventing

In [ ]:
df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track2.csv') ###
track2_list = df['x'].tolist() ###
track2_list_hashed = [] ###
for h in track2_list: ###
    h = '#'+h
    track2_list_hashed.append(h) ###
    
print track2_list_hashed ###

track2_statuses = [] ###

i = 0

for tag in track2_list_hashed: ###
    
#     i = i+1
    
#     if i == 30:
#         print "Issued approximately 60 queries, sleeping for 15 minutes."
#         print "Start : %s" % time.ctime()
#         time.sleep(900)
#         print "End : %s" % time.ctime()
#         i=1

    
    statuses = search_and_stream(tag, 300)
    track2_statuses += statuses ###
    
print "TOTAL number of statuses sampled:%i"%len(track2_statuses) ###

client = pymongo.MongoClient()
db = client.sn_sp
collection = db.net_0

for tweet_doc in track2_statuses: ###
    tweet_doc['track'] = 'track2' ###
    tweet_doc['_id'] = tweet_doc["id"]
    del tweet_doc['id']
    try:
        print "Saving tweet."
        collection.insert_one(tweet_doc)
    except:
        print "Preventing duplicate, skipping tweet and continuing."
        
client = pymongo.MongoClient()
collection1 = db.net_0
collection2 = db.net_00
                                    ###
cursor = collection1.find({'track':'track2'}, {'_id':'true', 'text':'true', 'entities.hashtags.text':'true', 'retweet_count':'true', 'track':'true'})

items_list = [ item for item in cursor ] 

for tweet_doc in items_list:
    hashtags = []
    for h in tweet_doc['entities']['hashtags']:
        h_lowered = h['text'].lower()
        hashtags.append(h_lowered)
    tweet_doc['hashtags'] = hashtags
    del tweet_doc['entities']
    try:
        print "Transfering tweet."
        collection2.insert_one(tweet_doc)
    except:
        print "Preventing duplicate, skipping tweet and continuing."

['#\xe3\x83\xaa\xe3\x82\xad\xe3\x83\x83\xe3\x83\x89', '#vapestoreworldwide', '#vapestagram', '#vaporize', '#galaxy', '#dablife', '#bong', '#smoke', '#funny', '#high', '#hotbox', '#legalisation', '#legalise', '#munchies', '#plants', '#indica', '#pa', '#music', '#news', '#radio', '#sport', '#cvccrew', '#ukvapers', '#eliquids', '#vapepics', '#vgod', '#handcheck', '#vapedaily', '#vapers', '#\xe3\x83\xa1\xe3\x83\xb3\xe3\x82\xbd\xe3\x83\xbc\xe3\x83\xab', '#rta', '#\xe3\x82\xa2\xe3\x83\x88\xe3\x83\x9e\xe3\x82\xa4\xe3\x82\xb6\xe3\x83\xbc', '#\xe3\x82\xbf\xe3\x83\xb3\xe3\x82\xaf', '#\xe3\x82\xbf\xe3\x83\x90\xe3\x82\xb3', '#\xe3\x83\x95\xe3\x83\xab\xe3\x83\xbc\xe3\x83\x84', '#\xe3\x83\xac\xe3\x83\x93\xe3\x83\xa5\xe3\x83\xbc', '#\xe3\x82\xb7\xe3\x83\xa7\xe3\x83\x83\xe3\x83\x97', '#\xe6\xb5\xb7\xe5\xa4\x96', '#\xe3\x83\x93\xe3\x83\xab\xe3\x83\x89', '#\xe5\x88\x9d\xe5\xbf\x83\xe8\x80\x85', '#vape_village', '#\xe7\xa6\x8f\xe5\xb2\xa1', '#unicig', '#beyondvapejapan', '#harajyuku', '#origami', '#toron

Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#リキッド' ... ... 


Found 	 205 	 statuses.
Sleeping for 16 seconds ... ...
Start : Tue Aug 30 10:51:01 2016


In [4]:
# df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/track3.csv') ###
# track3_list = df['x'].tolist() ###
# track3_list_hashed = [] ###
# for h in track3_list: ###
#     h = '#'+h
#     track3_list_hashed.append(h) ###
    
# print track3_list_hashed ###

# track3_statuses = [] ###

# i = 0

# for tag in track3_list_hashed: ###
    
# #     i = i+1
    
# #     if i == 30:
# #         print "Issued approximately 60 queries, sleeping for 15 minutes."
# #         print "Start : %s" % time.ctime()
# #         time.sleep(900)
# #         print "End : %s" % time.ctime()
# #         i=1

    
#     statuses = search_and_stream(tag, 800)
#     track3_statuses += statuses ###
    
# print "TOTAL number of statuses sampled:%i"%len(track3_statuses) ###

# client = pymongo.MongoClient()
# db = client.sn_sp
# collection = db.net_0

# for tweet_doc in track3_statuses: ###
#     tweet_doc['track'] = 'track3' ###
#     tweet_doc['_id'] = tweet_doc["id"]
#     del tweet_doc['id']
#     try:
#         print "Saving tweet."
#         collection.insert_one(tweet_doc)
#     except:
#         print "Preventing duplicate, skipping tweet and continuing."
        
# client = pymongo.MongoClient()
# collection1 = db.net_0
# collection2 = db.net_00
#                                     ###
# cursor = collection1.find({'track':'track3'}, {'_id':'true', 'text':'true', 'entities.hashtags.text':'true', 'retweet_count':'true', 'track':'true'})

# items_list = [ item for item in cursor ] 

# for tweet_doc in items_list:
#     hashtags = []
#     for h in tweet_doc['entities']['hashtags']:
#         h_lowered = h['text'].lower()
#         hashtags.append(h_lowered)
#     tweet_doc['hashtags'] = hashtags
#     del tweet_doc['entities']
#     try:
#         print "Transfering tweet."
#         collection2.insert_one(tweet_doc)
#     except:
#         print "Preventing duplicate, skipping tweet and continuing."

['#\xe7\x88\x86\xe7\x85\x99', '#vapemodels', '#ilsantonedellosvapo', '#cloud', '#paragon', '#electro', '#erig', '#cloudvapes', '#dipstick', '#again', '#drdabber', '#sourcevapes', '#boost', '#cloudpen', '#dabbing', '#furries', '#furry', '#fursuit', '#fursuiting', '#flosstradamus', '#orb', '#autofem', '#rap', '#pop', '#porto', '#prague', '#maroc', '#suisse', '#casablanca', '#foggymonster', '#vapepromo', '#deal', '#ukvaper', '#rda', '#vapegirls', '#vapeworld', '#vapemail', '#englandvapes', '#ukvapecommunity', '#vaporizers', '#ejuices', '#labordaysale', '#coupons', '#v2', '#augustbankholiday', '#capetown', '#competitions', '#epicesense', '#esense', '#slider', '#coilart', '#parallelcoil', '#tigercoil', '#vap', '#h', '#coi', '#vaporgram', '#dripclub', '#vapergram', '#able', '#ezdripper', '#avlyfe', '#vapehandcheck', '#ezbcwin', '#vapelove', '#vapelifestyle', '#meme', '#vapememe', '#atomizer', '#daily_dripper', '#iqos', '#\xe3\x82\xa2\xe3\x82\xa4\xe3\x82\xb3\xe3\x82\xb9', '#\xe3\x83\x8b\xe3\x

Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#爆煙' ... ... 


Found 	 17 	 statuses.
Sleeping for 3 seconds ... ...
Start : Tue Aug 30 00:50:03 2016
End : Tue Aug 30 00:50:06 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#vapemodels' ... ... 


Found 	 44 	 statuses.
Sleeping for 3 seconds ... ...
Start : Tue Aug 30 00:50:08 2016
End : Tue Aug 30 00:50:11 2016


...

Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#suisse' ... ... 


Found 	 99 	 statuses.
Sleeping for 3 seconds ... ...
Start : Tue Aug 30 00:52:50 2016
End : Tue Aug 30 00:52:53 2016


Succesfully connected to Twitter SEARCH API ... ... 
Searching for '#casablanca' ... ... 


Found 	 91 	 statuses.
Sleeping for 3 seconds ... ...
Start : Tue Aug 30 00:52:55 2016
End : Tue Aug 30 00:52:58 2016
Issued approximately 60 queries, sleeping for 15 minutes.
Start : Tue Aug 30 00:52:58 2016


KeyboardInterrupt: 

#### Count total tweets collected for each hashtag

In [27]:
client = pymongo.MongoClient()
db = client.sn_sp
collection = db.net_1



df = pd.read_csv('~/Desktop/twitter_data_analysis/develop/Py/tracked.csv')
df['tweets_count'] = pd.Series(index=df.index)

for idx, h in enumerate(df['x']):
    print idx
    print h
    tweets_for_h = collection.find({"hashtags":h}).count()
    df['tweets_count'][idx]=tweets_for_h

df

0
ecig
1
vaping


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
vape
3
vaporizer
4
marijuana
5
sorrynotsorry
6
eliquid
7
ecigs
8
vapeporn
9
cannabis
10
vaper
11
vapor
12
フレーバー
13
電子タバコ
14
ejuice
15
vapelife
16
vapefam
17
vapotage
18
vapoteurs
19
ecigarette
20
vapecommunity
21
vapelyfe
22
vapeon
23
subohm
24
tobaccocontrol
25
alert
26
free
27
giveaway
28
vapenation
29
vapeshop
30
vapehooligans
31
smoking
32
dabs
33
notblowingsmoke
34
kandypens
35
legalizeit
36
vapepen
37
mmj
38
pot
39
weed
40
cbd
41
sativa
42
israel
43
investing
44
beverlyhills
45
queen
46
mme
47
vmas
48
doinghappysnoopydance
49
itsmyfriday
50
aldubsilanabaay
51
40likes
52
class
53
country
54
quoteoftheday
55
sand
56
teen
57
timylovebegins
58
oops
59
teacher5adayslowchat
60
blessed
61
mod
62
vapeuk
63
ukvape
64
calivapers
65
cloudchaser
66
coil
67
smokefree
68
quitsmoking
69
cloudchasing
70
ecigarettes
71
fda
72
reddit
73
instavape
74
coilporn
75
legalize
76
hemp
77
stoner
78
breakthetaboo
79
endourpain
80
breakingnews
81
marijuanastocks
82
weedstocks
83
リキッド
84
vapestoreworldwide

,Unnamed: 0,x,tweets_count
0,1,ecig,1170.0
1,2,vaping,2075.0
2,3,vape,4202.0
3,4,vaporizer,485.0
4,5,marijuana,1765.0
5,6,sorrynotsorry,847.0
6,7,eliquid,816.0
7,8,ecigs,806.0
8,9,vapeporn,1311.0
9,10,cannabis,3143.0


In [37]:
df

df.to_csv('~/Desktop/twitter_data_analysis/develop/Py/tracked_tweets_count.csv')

### Move documents in mongo
Collect all the tweets in "net_0". Whe a satisfactory number for eachone is reached move into "net_1" computing the sentiment and keeping only __the relevant keys__:
- id
- text
- entities.hashtags.text
- retweet_count
- track
- sentiment
Move then to "net_2" transforming the hashtags to simp;e list for easier manipualation.

Move again to "hashtags_1" isolating what we waant to engineer for each hashtag and then to "hashtags_2" to eangineer the features.